<a href="https://colab.research.google.com/github/cytoscape/cytoscape-automation/blob/master/for-scripters/Python/visualizing-expression-data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visualizing Expression Data
## Yihang Xin and Alex Pico
## 2020-11-10

Probably the most common use of expression data in Cytoscape is to set the visual attributes of the nodes in a network according to expression data. This creates a powerful visualization, portraying functional relation and experimental response at the same time. Here, we will walk through the steps for doing this.

# Installation
The following chunk of code installs the `py4cytoscape` module.

In [1]:
%%capture
!python3 -m pip install python-igraph requests pandas networkx
!python3 -m pip install py4cytoscape
!python3 -m pip install wget

If you are using a remote notebook environment such as Google Colab, please execute the cell below. (If you're running on your local notebook, you don't need to do that.)



In [2]:
import requests
exec(requests.get("https://raw.githubusercontent.com/cytoscape/jupyter-bridge/master/client/p4c_init.py").text)
IPython.display.Javascript(_PY4CYTOSCAPE_BROWSER_CLIENT_JS) # Start browser client

Loading Javascript client ... e1bce960-aead-4d8b-a744-02b11cb20140 on https://jupyter-bridge.cytoscape.org
ADVICE: WHEN RUNNING UNDER COLAB, DO NOT RE-RUN THIS CELL WITHOUT MANUALLY EXECUTING Runtime | Factory Reset Runtime FROM THE COLAB MENU FIRST.


<IPython.core.display.Javascript object>

# Prerequisites
In addition to this package (py4cytoscape version 0.0.9), you will need:

Latest version of Cytoscape, which can be downloaded from https://cytoscape.org/download.html. Simply follow the installation instructions on screen.

Complete installation wizard

Install the filetransfer app from https://apps.cytoscape.org/apps/filetransfer

You can also install app inside Python notebook by running "py4cytoscape.install_app('Your App')"

# Import the required package

In [3]:
import os
import sys
import wget
import py4cytoscape as p4c

In [4]:
# Check Version
p4c.cytoscape_version_info()

{'apiVersion': 'v1',
 'automationAPIVersion': '1.2.0',
 'cytoscapeVersion': '3.9.0',
 'jupyterBridgeVersion': '0.0.2',
 'py4cytoscapeVersion': '0.0.9'}

# Loading Network

* Download the demo session and open the demo session using

In [5]:
# Download file to the local drive, if running in the cloud, download file to the cloud drive
cys_url = "http://nrnb.org/data/galFilteredSimpleData.cys"
file_name_local = wget.download(cys_url)

In [6]:
run_in_cloud = p4c.check_running_remote()

In [7]:
# if running in the cloud, send data to the local Cytoscape
if run_in_cloud:
  file_name_cloud = p4c.sandbox_send_to("galFilteredSimpleData.cys", "galFilteredSimpleData.cys")['filePath']

In [8]:
if run_in_cloud:
  abs_file_name = os.path.abspath(file_name_cloud)
else:
  abs_file_name = os.path.abspath(file_name_local)
abs_file_name

'/Users/yxin/CytoscapeConfiguration/filetransfer/default_sandbox/galFilteredSimpleData.cys'

In [9]:
session_cmd_list = ['session','open','file="',abs_file_name,'"']
session_cmd = " ".join(session_cmd_list)

In [10]:
p4c.commands.commands_run(session_cmd)

[]

* When the network first opens, the entire network is not visible because of the default zoom factor used. To see the whole network, we can use the fit_content function.

In [11]:
p4c.network_views.fit_content()

{}

# Label the Nodes
* Zoom in on the network so that node labels are visible.
* Check available column names.

In [12]:
p4c.tables.get_table_column_names('node')

['SUID',
 'shared name',
 'name',
 'selected',
 'COMMON',
 'gal1RGexp',
 'gal4RGexp',
 'gal80Rexp',
 'gal1RGsig',
 'gal4RGsig',
 'gal80Rsig']

* We are going to use the COMMON name attribute to give the nodes useful names:

In [13]:
p4c.style_mappings.set_node_label_mapping('COMMON')

style_name not specified, so updating "default" style.


''

* Verify that the node labels on the network have changed to their common names.

# Color the nodes
* Check available column names again

In [14]:
p4c.tables.get_table_column_names('node')

['SUID',
 'shared name',
 'name',
 'selected',
 'COMMON',
 'gal1RGexp',
 'gal4RGexp',
 'gal80Rexp',
 'gal1RGsig',
 'gal4RGsig',
 'gal80Rsig']

* Check the expression value range of gal80Rexp


In [15]:
gal80rexp_score_table = p4c.tables.get_table_columns('node', 'gal80Rexp')
gal80rexp_min = min(gal80rexp_score_table['gal80Rexp'])
gal80rexp_max = max(gal80rexp_score_table['gal80Rexp'])
print(gal80rexp_min)
print(gal80rexp_max)

-1.373
3.126


* The following produces a default gradient ranging from blue to red for expression values. Notice that the nodes in the network change color.

In [16]:
p4c.style_mappings.set_node_color_mapping('gal80Rexp', [gal80rexp_min, 0, gal80rexp_max], ['#0000FF', '#FFFFFF', '#FF0000'])

style_name not specified, so updating "default" style.


''

# Set the Default Node Color
Note that the default node color of pale blue is close to the color range in the created palette. A useful trick is to choose a color outside this spectrum to distinguish nodes with no defined expression value and those with slight repression.

* Here we choose a dark gray color for default node color.

In [17]:
p4c.style_defaults.set_node_color_default('#A9A9A9')

style_name not specified, so updating "default" style.


''

* Zoom out on the network view to verify that a few nodes have been colored gray.


In [18]:
p4c.network_views.fit_content()

{}

# Set the Node Shape
We imported both expression measurement values and corresponding significance values. We can use the significance values to change the shape of the nodes so that measurements we have confidence in appear as squares while potentially bad measurements appear as circles.

The following code maps the node with gal80Rsig value less than 0.05 to RECTANGLE shape, with greater than 0.05 to ELLIPSE shape.

In [19]:
p4c.style_mappings.update_style_mapping('default', p4c.style_mappings.map_visual_property('node shape', 'gal80Rsig', 'c', [0, 0.05, 1.00], ['rectangle', 'rectangle', 'ellipse']))

''

* Verify that some nodes now have a square shape.

# Fun with Charts
In addition to coloring the nodes, Cytoscape also provides the ability to draw charts and graphs on each node. For example, suppose we wanted to display a bar chart showing all of the expression values on each of our nodes?

* To reset things a little, remove the mapping for Fill Color by

In [20]:
p4c.styles.get_visual_property_names()

['COMPOUND_NODE_PADDING',
 'COMPOUND_NODE_SHAPE',
 'DING_RENDERING_ENGINE_ROOT',
 'EDGE',
 'EDGE_BEND',
 'EDGE_CURVED',
 'EDGE_LABEL',
 'EDGE_LABEL_COLOR',
 'EDGE_LABEL_FONT_FACE',
 'EDGE_LABEL_FONT_SIZE',
 'EDGE_LABEL_ROTATION',
 'EDGE_LABEL_TRANSPARENCY',
 'EDGE_LABEL_WIDTH',
 'EDGE_LINE_TYPE',
 'EDGE_PAINT',
 'EDGE_SELECTED',
 'EDGE_SELECTED_PAINT',
 'EDGE_SOURCE_ARROW_SELECTED_PAINT',
 'EDGE_SOURCE_ARROW_SHAPE',
 'EDGE_SOURCE_ARROW_SIZE',
 'EDGE_SOURCE_ARROW_UNSELECTED_PAINT',
 'EDGE_STACKING',
 'EDGE_STACKING_DENSITY',
 'EDGE_STROKE_SELECTED_PAINT',
 'EDGE_STROKE_UNSELECTED_PAINT',
 'EDGE_TARGET_ARROW_SELECTED_PAINT',
 'EDGE_TARGET_ARROW_SHAPE',
 'EDGE_TARGET_ARROW_SIZE',
 'EDGE_TARGET_ARROW_UNSELECTED_PAINT',
 'EDGE_TOOLTIP',
 'EDGE_TRANSPARENCY',
 'EDGE_UNSELECTED_PAINT',
 'EDGE_VISIBLE',
 'EDGE_WIDTH',
 'EDGE_Z_ORDER',
 'NETWORK',
 'NETWORK_ANNOTATION_SELECTION',
 'NETWORK_BACKGROUND_PAINT',
 'NETWORK_CENTER_X_LOCATION',
 'NETWORK_CENTER_Y_LOCATION',
 'NETWORK_CENTER_Z_LOCATION

In [21]:
p4c.style_mappings.delete_style_mapping("default", "NODE_FILL_COLOR")

''

* Now change the default value to a lighter shade of grey so we can see our chart.


In [22]:
p4c.style_defaults.set_node_color_default('#D3D3D3')

style_name not specified, so updating "default" style.


''

* The following indicates that we’re going to use the data from these three columns expression data (gal1RGexp, gal4RGexp, gal80Rexp) to create bar chart.

In [23]:
p4c.style_defaults.set_node_custom_bar_chart(["gal1RGexp", "gal4RGexp", "gal80Rexp"])

style_name not specified, so updating "default" style.


''